In [1]:
%load_ext watermark
%watermark -d -v -m -p pandas

2017-05-05 

CPython 3.5.2
IPython 5.1.0

pandas 0.18.1

compiler   : GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)
system     : Darwin
release    : 16.5.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit


## Grab data from OASI web database

The [OASI web database](http://www.ti.ch/oasi) is an observatory platform for the environmental data (Osservatorio Ambientale della Svizzera Italiana). This database stores data from air quality to meteo, energy, noise, traffic and so on.
Each data is accessible by a location (a measure point) and a parameter, dovoded in yearly, monthly, daily and hourly measurment. The limit consist that each database (csv file) is for an unique location and a unique parameter. For example, if we want to get data for PM10, PM2.5, O2, NO3 for Lugano, annualy, we have to download 4 different files. Furthermore, ther is a limit that prevents to download more than 20 days of hourly datas and more than 1 year of daily data. 

The following code allow to concatenate all the parameter for all location for each resolution (day, month or year)

In [2]:
import pandas as pd
from functools import reduce

from_date = "2007-01-01"
to_date = "2017-04-30"

parameter = ['pm10','pm2.5','o3','no2']

# Airolo, Biasca, Bioggio, Bodio, Brione s. Minusio, Camignolo, Chiasso, Giubiasco, Locarno, Lugano, Magadino, Mendrisio, Pregassona, San Vittore, Vezia
location = ['auto_275', '511%2002%2008', '304%2001%2005', '406%2001%2003', '525%2001%2001', '610%2002%2006',
           'auto_51', '423%2002%2002', 'Nabel_LUG', 'Nabel_MAG', '614%2001%2003', '596%2001%2001', 'auto_75', '591%2001%2001']
city = ['Airolo', 'Bioggio', 'Bodio', 'Bione s. Minusio', 'Camignolo', 
        'Chiasso', 'Giubiasco', 'Locarno', 'Lugano', 'Magadino', 'Mendrisio',
        'Pregassona', 'San Vittore', 'Vezia']

### Dati giornalieri

For daily data, the start date has been fixed to 2009 since auto_51 (Giubiasco) has measurments from 1.10.2008. auto_275 (Airolo) station has been dropped from de location list since it has only measurments starting from 2016. 

In [3]:
from_date_d = "2009-01-01"
to_date_d = "2017-04-30"

from_index = pd.date_range(from_date_d , to_date_d , freq="AS")
to_index = pd.date_range(from_date_d , to_date_d , freq="A")

df_from = pd.DataFrame({"From": from_index})
from_data_d = pd.to_datetime(df_from['From']).dt.strftime('%Y-%m-%d').tolist()
df_to = pd.DataFrame({"To": to_index})
to_data_d = pd.to_datetime(df_to['To']).dt.strftime('%Y-%m-%d').tolist()

location = ['511%2002%2008', '304%2001%2005', '406%2001%2003', '525%2001%2001', '610%2002%2006',
           'auto_51', '423%2002%2002', 'Nabel_LUG', 'Nabel_MAG', '614%2001%2003', '596%2001%2001', 'auto_75', '591%2001%2001']
city = ['Bioggio', 'Bodio', 'Bione s. Minusio', 'Camignolo', 
        'Chiasso', 'Giubiasco', 'Locarno', 'Lugano', 'Magadino', 'Mendrisio',
        'Pregassona', 'San Vittore', 'Vezia']


resolution = 'd' # daily resolution
urls = []
dfs = []
# Base URL
CSV_URL = 'http://www.oasi.ti.ch/web/rest/measure/csv?domain=air&resolution={}&parameter={}&from={}&to={}&location={}'
# from_date = '2016-04-30'
# to_date = '2017-04-30'

for l in location:
    dfs1 = []
    for p in parameter:
        dfs2 = []
        for fd, td in zip(from_data_d, to_data_d):
            url = CSV_URL.format(resolution, p, fd, td, l)
#             print(url)
            df = pd.read_csv(url, comment='#', sep=';', usecols=[0, 1], index_col='data')
            dfs2.append(df)
        dfs1.append(pd.concat(dfs2, axis=0))
    dfs.append(pd.concat(dfs1, axis=1))

air_day = pd.concat(dfs, keys=city).rename_axis(('city','data')).reset_index()
# .dropna(axis=1, how='all')
air_day.to_csv('air_day.csv', index=False)

### Dati mensili

In [4]:
resolution = 'm' # risoluzione mensile
urls = []
dfs = []
CSV_URL = 'http://www.oasi.ti.ch/web/rest/measure/csv?domain=air&resolution={}&parameter={}&from={}&to={}&location={}'


for l in location:
    dfs1 = []
    for p in parameter:
        url = CSV_URL.format(resolution, p, from_date, to_date, l)
        df = pd.read_csv(url, comment='#', sep=';', usecols=[0, 1], index_col='data')
        dfs1.append(df)
    dfs.append(pd.concat(dfs1, axis=1))

air_month = pd.concat(dfs, keys=city).rename_axis(('city','data')).reset_index()
# .dropna(axis=1, how='all')
air_month.to_csv('air_month.csv', index=False)

### Dati annuali

In [5]:
resolution = 'y' # risoluzione mensile
urls = []
dfs = []
CSV_URL = 'http://www.oasi.ti.ch/web/rest/measure/csv?domain=air&resolution={}&parameter={}&from={}&to={}&location={}'


for l in location:
    dfs1 = []
    for p in parameter:
        url = CSV_URL.format(resolution, p, from_date, to_date, l)
        df = pd.read_csv(url, comment='#', sep=';', usecols=[0, 1], index_col='data')
        dfs1.append(df)
    dfs.append(pd.concat(dfs1, axis=1))

air_year = pd.concat(dfs, keys=city).rename_axis(('city','data')).reset_index()
# .dropna(axis=1, how='all')
air_year.to_csv('air_year.csv', index=False)